In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#!pip install -r requirements.txt

In [3]:
#!pip install langchain_chroma

In [4]:
#!pip install langchain_huggingface

In [ ]:
import os
from dotenv import load_dotenv
from  langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

import pandas as pd
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


import pandas as pd

from langchain.vectorstores import Chroma
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Load the data
sampled_df = pd.read_csv("./sampled_data.csv")



# Select only the 'question' and 'sql_query' columns
df_selected = sampled_df[['question', 'sql_query']]

# Save the new DataFrame to a new CSV file
df_selected.to_csv('data.csv', index=False)

In [ ]:
# Load the data
sampled_df = pd.read_csv("./data.csv")
# Split the data into training (70%), validation (15%), and test (15%) sets
train_df, temp_df = train_test_split(sampled_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)



# Show column names and sample data
print("Column Names:")
print(sampled_df.columns)

Column Names:
Index(['question', 'sql_query'], dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Tokenize the SQL queries and questions
sql_tokenizer = Tokenizer()
sql_tokenizer.fit_on_texts(train_df['sql_query'])

question_tokenizer = Tokenizer()
question_tokenizer.fit_on_texts(train_df['question'])

# Convert text to sequences
train_sql_seq = sql_tokenizer.texts_to_sequences(train_df['sql_query'])
train_question_seq = question_tokenizer.texts_to_sequences(train_df['question'])

val_sql_seq = sql_tokenizer.texts_to_sequences(val_df['sql_query'])
val_question_seq = question_tokenizer.texts_to_sequences(val_df['question'])

test_sql_seq = sql_tokenizer.texts_to_sequences(test_df['sql_query'])
test_question_seq = question_tokenizer.texts_to_sequences(test_df['question'])

# Pad sequences to ensure uniform input size
max_sql_len = max(len(seq) for seq in train_sql_seq)
max_question_len = max(len(seq) for seq in train_question_seq)

train_sql_seq = pad_sequences(train_sql_seq, maxlen=max_sql_len, padding='post')
train_question_seq = pad_sequences(train_question_seq, maxlen=max_question_len, padding='post')

val_sql_seq = pad_sequences(val_sql_seq, maxlen=max_sql_len, padding='post')
val_question_seq = pad_sequences(val_question_seq, maxlen=max_question_len, padding='post')

test_sql_seq = pad_sequences(test_sql_seq, maxlen=max_sql_len, padding='post')
test_question_seq = pad_sequences(test_question_seq, maxlen=max_question_len, padding='post')


**Build the Model**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the model
embedding_dim = 256
latent_dim = 512

# Encoder
encoder_inputs = Input(shape=(max_sql_len,))
enc_emb = Embedding(input_dim=len(sql_tokenizer.word_index) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_question_len,))
dec_emb_layer = Embedding(input_dim=len(question_tokenizer.word_index) + 1, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(len(question_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



**Train the Model**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Tokenize the SQL queries and questions
sql_tokenizer = Tokenizer()
sql_tokenizer.fit_on_texts(train_df['sql_query'])

question_tokenizer = Tokenizer()
question_tokenizer.fit_on_texts(train_df['question'])

# Convert text to sequences
train_sql_seq = sql_tokenizer.texts_to_sequences(train_df['sql_query'])
train_question_seq = question_tokenizer.texts_to_sequences(train_df['question'])

val_sql_seq = sql_tokenizer.texts_to_sequences(val_df['sql_query'])
val_question_seq = question_tokenizer.texts_to_sequences(val_df['question'])

test_sql_seq = sql_tokenizer.texts_to_sequences(test_df['sql_query'])
test_question_seq = question_tokenizer.texts_to_sequences(test_df['question'])

# Determine max sequence lengths
max_sql_len = max(max(len(seq) for seq in train_sql_seq), max(len(seq) for seq in val_sql_seq), max(len(seq) for seq in test_sql_seq))
max_question_len = max(max(len(seq) for seq in train_question_seq), max(len(seq) for seq in val_question_seq), max(len(seq) for seq in test_question_seq))

# Pad sequences to ensure uniform input size
train_sql_seq = pad_sequences(train_sql_seq, maxlen=max_sql_len, padding='post')
train_question_seq = pad_sequences(train_question_seq, maxlen=max_question_len, padding='post')

val_sql_seq = pad_sequences(val_sql_seq, maxlen=max_sql_len, padding='post')
val_question_seq = pad_sequences(val_question_seq, maxlen=max_question_len, padding='post')

test_sql_seq = pad_sequences(test_sql_seq, maxlen=max_sql_len, padding='post')
test_question_seq = pad_sequences(test_question_seq, maxlen=max_question_len, padding='post')

# Prepare decoder input and target sequences
train_decoder_input_data = train_question_seq[:, :-1]
train_target_data = train_question_seq[:, 1:]

val_decoder_input_data = val_question_seq[:, :-1]
val_target_data = val_question_seq[:, 1:]

test_decoder_input_data = test_question_seq[:, :-1]
test_target_data = test_question_seq[:, 1:]

# Ensure the target data has the correct shape for the model's output
train_target_data = train_target_data[..., None]
val_target_data = val_target_data[..., None]
test_target_data = test_target_data[..., None]


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the model
embedding_dim = 256
latent_dim = 512

# Encoder
encoder_inputs = Input(shape=(max_sql_len,))
enc_emb = Embedding(input_dim=len(sql_tokenizer.word_index) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_question_len - 1,))
dec_emb_layer = Embedding(input_dim=len(question_tokenizer.word_index) + 1, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(len(question_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(
    [train_sql_seq, train_decoder_input_data],
    train_target_data,
    batch_size=64,
    epochs=50,
    validation_data=([val_sql_seq, val_decoder_input_data], val_target_data)
)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 35)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 41)]                 0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 35, 256)              1127680   ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 41, 256)              1227520   ['input_4[0][0]']             
                                                                                            

final validation loss < then the final train loss value ==>Overfit

the last final train loss< than last final validation loss
==>Overfit

In [ ]:
import pickle

# Retrieve the trained model from history
model = history.model

# Save the trained model
model.save('model.h5')



In [ ]:

# Save the tokenizer objects
with open('sql_tokenizer.pickle', 'wb') as handle:
    pickle.dump(sql_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('question_tokenizer.pickle', 'wb') as handle:
    pickle.dump(question_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def decode_sequence(input_seq):
    # Define the encoder model
    encoder_model = Model(inputs=model.input[0], outputs=model.get_layer('lstm_2').output[1:])

    # Define the decoder model
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = model.get_layer('lstm_3')(
        model.get_layer('embedding_3')(model.input[1]),
        initial_state=decoder_states_inputs
    )
    decoder_states = [state_h, state_c]
    decoder_outputs = model.get_layer('dense_1')(decoder_outputs)
    decoder_model = Model(
        [model.input[1]] + decoder_states_inputs,
        [decoder_outputs] + decoder_states
    )

    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Populate the first token of target sequence with the start token.
    start_token_index = question_tokenizer.word_index['starttoken']
    target_seq[0, 0] = start_token_index

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token within the vocabulary size
        sampled_token_index = np.random.choice(len(output_tokens[0, -1, :]), p=output_tokens[0, -1, :])
        sampled_word = question_tokenizer.index_word.get(sampled_token_index, '')  # Get the word corresponding to the index

        # Exit condition: either hit max length or find stop token.
        if (sampled_word == 'endtoken' or len(decoded_sentence.split()) >= max_question_len):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()


In [4]:
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the trained model
model = load_model('model.h5')

# Load the tokenizer objects
with open('sql_tokenizer.pickle', 'rb') as handle:
    sql_tokenizer = pickle.load(handle)


# Load the tokenizer objects
with open('question_tokenizer.pickle', 'rb') as handle:
    question_tokenizer = pickle.load(handle)

# Manually add 'starttoken' to the word index
question_tokenizer.word_index['starttoken'] = len(question_tokenizer.word_index) + 1


# Example SQL query
input_sql_query = "SELECT Name, Composer, UnitPrice FROM tracks LIMIT 10"

# Tokenize and pad the input SQL query
input_sql_seq = sql_tokenizer.texts_to_sequences([input_sql_query])
input_sql_seq = pad_sequences(input_sql_seq, maxlen=max_sql_len, padding='post')

# Generate natural language question
decoded_question = decode_sequence(input_sql_seq)
print("Generated Question:", decoded_question)
